# Challenge for LUD22

In [6]:
import pandas as pd
import numpy as np
from rec import svd_decompose, svd_recommend_to_list
from rec import inter_matr_binary, split_interactions
from rec import recTopKPop
from rec import get_user_item_score

path = 'data_challenge/'

usr_path = path + 'MRSC_2022_demo.txt'
itm_path = path + 'MRSC_2022_tracks.txt'
inter_path = path + 'MRSC_2022_inter.txt'
target_usr_path = path + 'MRSC_2022_target_users.txt'

In [7]:

df_users = pd.read_csv(usr_path, sep='\t', header=None, names=['location', 'age', 'gender', 'date'])
df_items = pd.read_csv(itm_path, sep='\t', header=None, names=['artist', 'track'])

inter_matrix = inter_matr_binary(usr_path, itm_path, inter_path)


# Hypertuning Parameters

In [8]:
### Hypertuning
top_k = 15  #recommendations
n = 5  #amount of neighbors to consider

# Locating Users

In [9]:
users = np.loadtxt(target_usr_path, dtype=int)

# Testing user clustering

In [12]:
inter_matrix

array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [15]:
def calc_pred(row1, row_2, counter_, rec_row):
    for i, elem in enumerate(row_2):
        if counter_ == 15:
            break
        if elem == 1 and row1[i] != 1 and ((i not in rec_row) or i ==1 or i== 0):
            rec_row[counter_] = i
            counter_ += 1

    return rec_row, counter_

sums = np.empty(shape=inter_matrix.shape[0])
rec = np.full((len(users), top_k), -1)

for k, user in enumerate(users):
    counter = 0
    threshold = 0.4
    while counter != 15:
        print(f"Counter:{counter}, iterations_user:{k}, Total_users:{len(users)} ")

        threshold *= 0.75

        for ii, row2 in enumerate(inter_matrix):
            if user == ii:
                continue

            if np.corrcoef(inter_matrix[user], row2)[0, 1] > threshold:
                rec[k], counter = calc_pred(inter_matrix[user], row2, counter, rec[k])

                if counter == 15:
                    break
    print(rec[k])
    print(f"user {k} is done")

Counter:0, iterations_user:0, Total_users:1299 
Counter:0, iterations_user:0, Total_users:1299 
[ 256  383  544  556 5787  479  862 1307 3009 4624 5063   28   33  309
  310]
user 0 is done
Counter:0, iterations_user:1, Total_users:1299 
[ 270  271  295  332  788 6893  317  494 1025 1238 4996  269  273 2346
 5323]
user 1 is done
Counter:0, iterations_user:2, Total_users:1299 
Counter:5, iterations_user:2, Total_users:1299 
[1267 1964 2435    1 2233   39  219  226  230  236  295  305  343  795
  796]
user 2 is done
Counter:0, iterations_user:3, Total_users:1299 
Counter:0, iterations_user:3, Total_users:1299 
Counter:8, iterations_user:3, Total_users:1299 
[ 179  230  352  762 2303 8171 8531  487  618 1073 1074 1075 1076  456
  503]
user 3 is done
Counter:0, iterations_user:4, Total_users:1299 
[  65   90  162  428  551  582  708  781  872 1527 1595 6107  395  689
 1602]
user 4 is done
Counter:0, iterations_user:5, Total_users:1299 
Counter:0, iterations_user:5, Total_users:1299 
[ 418  

In [17]:
print(rec.shape)

(1299, 15)


# Writing into file

In [18]:
with open('rec_k12006268_Viktor_loreth.tsv', 'w') as f:
    for k, row in enumerate(rec):
        f.write(f'{users[k]}\t')
        for ele in row:
            f.write(f'{int(ele)}')
            if ele != row[-1]:
                f.write(f',')

        f.write('\n')



[[1363 2256  189 ... 1913  237  247]
 [1078  857  919 ...  431   97  344]
 [ 403  627  661 ... 2098 2562 2554]
 ...
 [ 889  496  653 ...  710 1305 1639]
 [   3  187 1170 ... 1262  154 1028]
 [  28 1255 2689 ...  895 2076 2558]]
